<h1>Preparation of the English Twitter Data</h1>

<p>This file is for preparing the german tweets. The preparation is mainly for training my own model for sentiment detection with Flair. </p>

In [1]:
import csv
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from HanTa import HanoverTagger as ht

<p>Download der Stopwörter Listen</p>

In [2]:
#nltk.download('stopwords')

<h1>Einlesen und bestimmung der Felder</h1>
<p>Durch die weitern Operatoren, kann ich die einzelnen Felder in DF bestimmen. Durch die Lambda Funktion werden die Str zu Listen, was die spätere verarbeitung erleichtert. 
    Durch die Parse_dates Funktion wird das Datumsfeld zu einem Timestamp, was die weiter Verarbeitung wesentlich leichter macht.</p>

In [3]:
df_manager = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Raw_File/Manager_EN/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_manager = df_manager.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)

In [4]:
len(df_manager)

17844

<h3>Löschen von unnötigen Informationen</h3>

<p>Hiermit werden alle Textbausteine gelöscht, die nicht benötigt werden. Personen bleiben in diesem Schritt noch enthalten.</p>

In [5]:
#just for test
for each in range(len(df_manager)):
    df_manager.loc[df_manager["text"] == 
                      df_manager.loc[each]["text"], "text"] = re.sub(r'http\S+', '',(re.sub('[#|$|\r]', '', df_manager.loc[each]["text"])))

<h1>Löschen von nicht weiter relevanten Tweets</h1>
<h3>Löschen von Retweets</h3>
<p>Alle Retweets löschen, da diese in diesem Schritt nicht benötigt werden.</p>

In [6]:
counter = 0
retweet_list = []
for each in range(len(df_manager)):
    if "retweeted" in df_manager.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)

9473


In [7]:
df_manager = df_manager.drop(retweet_list)
df_manager = df_manager.reset_index(drop=True)
print(len(df_manager))

8371


<h3>Löschen von doppelten Tweets</h3>
<p>Alle Tweets, die dopplet im Datensatz enthalten sind, anhand der tweet_id werden gelöscht. Somit werden auch die zuvor aufgespalteten Listen wieder gelöscht</p>

In [8]:
df_manager = df_manager.drop_duplicates(subset=["tweet_id"])
df_manager = df_manager.reset_index(drop=True)
print(len(df_manager))

8328


<h3>Nicht Kontextbezogene Tweets löschen</h3>
<p>So kann ich prüfen ob bestimmte Begriffe in den Texten enthalten sind und mir die conversation_id ausgeben lassen. Diese lasse ich dann löschen und auch zugehörige Tweets, da diese nicht in den Kontext passen.<br>Die wörter in der Liste müssen klein geschrieben werden, da auch alle Buchstaben für die Prüfung klein geschrieben werden müssen, da sinst nicht alles erkannt wird.</p>
<p>Zu löschende Begriffe</p>
<ul>
    <li>meiko</li>
    <li>calisthenics</li>
    <li>pulled</li>
    <li>boxing</li>
    <li>fom</li>
    <li>dlr</li>
    <li>eurocity</li>
    <li>m.o.s.s</li>
    <li>markus_braun_</li>
</ul>

<p>Überprüfung der englischen Tweets. Im ersten Schritt wurden die Retweets gelöscht, dann alle doppelten Tweets. Anschließend gab es eine Liste mit Begriffen, die definitiv etwas mit dem Thema zu tun hatten. Diese wurden aus dem Datensatz zur prüfung gelöscht. Darauf folgte eine Liste mit Begriffen, die nichts mit dem Thema zu tun haben und deswegen gelöscht werden. Abschließend wurden zu überprüfungszwecken Tweets mit doppelten Texten gelöscht und im Anschluss einzelne Tweet ID gelöscht.</p>

In [9]:
#List with single IDs, which are not relevant for the dataset
list_tweet_id = [719940834028863489, 719940834028863489, 919891703116566529, 730315192559673344, 1085012780053336064, 748146758232879104,
                 1275383293504978945, 1121414883512393728, 1288961015351316480, 1284465825915838464, 955660777960230917, 962669158923153409,
                 1016245557021507584, 1074388350734340097, 1115926018051792898, 1132148944031457280, 1130223973701246976, 1174984360669278209,
                 1220356450809188353, 1255176912202108929]

#Check and get tweet_ids for tweets with the same text as in the list_tweet_id.
for tweet_id in list_tweet_id:
    for each in df_manager.loc[df_manager["text"] == df_manager.loc[df_manager["tweet_id"] == tweet_id]["text"].values[0]]["tweet_id"].values:
        if each not in list_tweet_id:
            list_tweet_id.append(each)
            
#List of words which does not have anything to do with wirecard.
not_used_words_list = ["meiko", "calisthenics", "pulled pork", "boxing", " fom", "dlr", "eurocity", "m.o.s.s", "markus_braun_"]

#Iterate though the dataframe to get each tweet_id and save it in the list_tweet_id
for each in range(len(df_manager)):
    for word in not_used_words_list:
        if word in df_manager.iloc[each]["text"].lower():
            if df_manager.iloc[each]["tweet_id"] not in list_tweet_id:
                list_tweet_id.append(df_manager.iloc[each]["tweet_id"])


#Get further tweet_ids for every tweet_id that is stored in list_tweet_id
for tweet in list_tweet_id:
    for each in df_manager.loc[(
        df_manager["tweet_id"] == tweet) | (
        df_manager["conversation_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["conversation_id"].values[0]) | (
        df_manager["referenced_tweet_id"] == tweet) | (
        (df_manager["referenced_tweet_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["referenced_tweet_id"].values[0]) & (df_manager["referenced_tweet_id"] != "None")) | (
        df_manager["referenced_tweet_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["conversation_id"].values[0])]["tweet_id"].values:
        #If the value is not in the list, it will be added to the list_tweet_id
        if each not in list_tweet_id:
            list_tweet_id.append(each)

<p>Löschen der zuvor ermitteletn Tweets.</p>

In [10]:
#Iterate through list_tweet_id and remove every tweet, that has nothing to do with Wirecard.
for each in list_tweet_id:
    df_manager = df_manager.drop(df_manager.loc[df_manager["tweet_id"] == each].index[0])
#Set back the index. Otherwise it will not be possible to work with the index any loner. 
df_manager = df_manager.reset_index(drop=True)

In [11]:
for each in range(len(df_manager)):
    df_manager.loc[df_manager["text"] == 
                      df_manager.loc[each]["text"], "text"] = re.sub('@[^\s]+', '', df_manager.loc[each]["text"])
print(len(df_manager))

8275


<h1>Hinzufügen weiterer Twitter Datensätze</h1>
<p>Hier werden die weiteren Datensätze zusammengeführt.</p>

<h2>Datensatz mit Abfrage nach Wirecard</h2>

In [12]:
df_wirecard = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Raw_File/Wirecard_EN/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_wirecard = df_wirecard.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)


In [13]:
for each in range(len(df_wirecard)):
    df_wirecard.loc[df_wirecard["text"] == 
                      df_wirecard.loc[each]["text"], "text"] = re.sub(r'http\S+', '', 
                                                                        (re.sub('@[^\s]+', '',
                                                                                re.sub('[#|$|\r]', '', 
                                                                                       df_wirecard.loc[each]["text"]))))
print(len(df_wirecard))

204188


<h3>Löschen von Retweets</h3>

In [14]:
counter = 0
retweet_list = []
for each in range(len(df_wirecard)):
    if "retweeted" in df_wirecard.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)
df_wirecard = df_wirecard.drop(retweet_list)
df_wirecard = df_wirecard.reset_index(drop=True)
print(len(df_wirecard))

95717
108471


<h3>Löschen von doppelten Einträgen</h3>

In [15]:
df_wirecard = df_wirecard.drop_duplicates(subset=["tweet_id"])
df_wirecard = df_wirecard.reset_index(drop=True)
print(len(df_wirecard))

108218


<h2>Abfrage nach Chashtag Wirecard</h2>

In [16]:
df_wrcdf = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Raw_File/WRCDF_EN/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_wrcdf = df_wrcdf.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)


In [17]:
for each in range(len(df_wrcdf)):
    df_wrcdf.loc[df_wrcdf["text"] == 
                      df_wrcdf.loc[each]["text"], "text"] = re.sub(r'http\S+', '', 
                                                                        (re.sub('@[^\s]+', '',
                                                                                re.sub('[#|$|\r]', '', 
                                                                                       df_wrcdf.loc[each]["text"]))))
print(len(df_wrcdf))

1260


<h3>Löschen von Retweets</h3>

In [18]:
counter = 0
retweet_list = []
for each in range(len(df_wrcdf)):
    if "retweeted" in df_wrcdf.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)
df_wrcdf = df_wrcdf.drop(retweet_list)
df_wrcdf = df_wrcdf.reset_index(drop=True)
print(len(df_wrcdf))

250
1010


<h3>Löschen von doppelten Tweets</h3>

In [19]:
df_wrcdf = df_wrcdf.drop_duplicates(subset=["tweet_id"])
df_wrcdf = df_wrcdf.reset_index(drop=True)
print(len(df_wrcdf))

1010


<h1>Zusammenführen der Datensätze</h1>

In [20]:
list_data_frames = [df_manager, df_wirecard, df_wrcdf]
df = pd.concat(list_data_frames, ignore_index=True)
print(len(df))

117503


<h3>Löschen der doppelten Posts</h3>
<p>Dadurch, dass die Abfrage durch mehrere Querys erfolgt ist, befinden sich einige Tweets doppelt in dem Datensatz. Diese werden hier anhand der <b>"tweet_id"</b> gelöscht. </p>

In [21]:
df = df.drop_duplicates(subset=["tweet_id"])
df = df.reset_index(drop=True)
print(len(df))

111199


<p>Der neue Datensatz wird auf plausibilität geprüft.</p>

<p><b>Fertig</b> mit der Vorverarbeitung der Daten in deutsch. Es ist nun ein Datensatz verfügbar mit allen Tweets für den deutschsprachigen Raum. Es sind keine doppelten Tweets enthalten, solange diese keine gleiche tweet_id enthalten. Auch Nutzer und tweets, die nichts zu dem eigentlichen Thema Wirecard beitragen sind nicht mehr im Datensatz enthalten. Vereinzelt können natürlich unnütze Tweets vorhanden sein, aber die Nutzer mit vielen Interaktionen sind überprüft und auch die Begriffe, die besonders oft vorkommen sind überprüft worden, ob diese etwas mit dem Thema zu tun haben.<br>Jetzt muss das gleich noch mit den Englischen tweets geschehen und dann müssen Datensätze zur Überprüfung ausgegeben werden.<br> Um den zufälligen Datensatz zu erstellen sollte ich noch einen zufallszahlengenerator nutzen, sodass ich möglichst zufällige Tweets zum testen erhalte. Hier scheint die Pandas Funktion <b>"sample"</b> gut geeignet zu sein. </p>

In [22]:
df.sample(n = 20)

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
43770,1177674365971976198,1095739951478697987,2019-09-27 20:00:23+00:00,1177674365971976198,en,0,0,0,0,Hootsuite Inc.,None,None,Talk CMO Wirecard Announces New Salesforce Co...
94112,1288266779962167302,2235329532,2020-07-29 00:15:09+00:00,1288266779962167302,en,0,0,0,0,Twitter Web App,None,None,"Wirecard scandal, Mastercard manager in trouble"
32813,1093506439950286848,2603184318,2019-02-08 12:24:26+00:00,1093848026240696320,en,0,0,0,0,Twitter Web Client,replied_to,1093847188004843520,it's not a raid.... Wirecard approached singa...
13457,770647378571456512,3044159284,2016-08-30 15:40:27+00:00,770647378571456512,en,0,0,0,0,wlst,None,None,Google cloud unit close to winning PayPal busi...
18248,850619364311269376,42056054,2017-04-08 08:00:34+00:00,850619364311269376,en,18,0,7,1,Agorapulse app,None,None,One of the largest drugstore chains in Germany...
45564,1184853974299693056,565832571,2019-10-17 17:36:54+00:00,1184886013824512000,en,0,1,2,0,Twitter Web App,replied_to,1184853974299693056,Dude. Nobody is lying. well Wirecard probab...
97374,1285504670136500225,320888254,2020-07-21 09:19:31+00:00,1285504670136500225,en,0,0,0,0,Twitter Web App,None,None,Wirecard ...
83134,1274905389298069505,961816966100017152,2020-06-22 04:07:08+00:00,1274916808693370881,en,0,0,0,0,Twitter for iPhone,replied_to,1274905389298069505,Not running well lufthansa Wirecard
66091,1276471590394478594,228550533,2020-06-27 00:00:27+00:00,1276666669402927105,en,0,1,0,0,Twitter Web App,replied_to,1276659403651977216,No one can know for sure. But I have a ver...
72002,1276198376451698688,66162566,2020-06-25 16:59:38+00:00,1276198376451698688,en,0,0,0,0,Twitter for iPhone,None,None,Wirecard - There's No Value Here. It's Dead An...


In [23]:
df.loc[df["tweet_id"] == 964197782876839936]["text"].values[0]

'“We use data-driven, innovative technologies so our clients can remove barriers between themselves and their customers.” This is WirecardUSA.    '

<h2>Finalen Datensatz als CSV ausgeben.</h2>

In [24]:
df.to_csv('Dataset_Wirecard.csv', sep=';', index=False)

<h2>Erstellen der CSV zur Überprüfung der Sentiment Ergebnisse</h2>

In [25]:
df_sentiment_validation = df.sample(n = 1500)[["tweet_id", "text"]]
batch = 14
counter = 0 
for batch in range(0,15):
    start = 100*batch
    end = 100*batch+100
    csv_name = 'en_validation_{count}.csv'.format(count=batch)
    df_sentiment_validation.iloc[start:end].to_csv(csv_name, sep=';', index=False)
    print("Start bei {start} und ende bei {ende}. Name der Datei: {csv}".format(start=start, ende=end, csv=csv_name))

Start bei 0 und ende bei 100. Name der Datei: en_validation_0.csv
Start bei 100 und ende bei 200. Name der Datei: en_validation_1.csv
Start bei 200 und ende bei 300. Name der Datei: en_validation_2.csv
Start bei 300 und ende bei 400. Name der Datei: en_validation_3.csv
Start bei 400 und ende bei 500. Name der Datei: en_validation_4.csv
Start bei 500 und ende bei 600. Name der Datei: en_validation_5.csv
Start bei 600 und ende bei 700. Name der Datei: en_validation_6.csv
Start bei 700 und ende bei 800. Name der Datei: en_validation_7.csv
Start bei 800 und ende bei 900. Name der Datei: en_validation_8.csv
Start bei 900 und ende bei 1000. Name der Datei: en_validation_9.csv
Start bei 1000 und ende bei 1100. Name der Datei: en_validation_10.csv
Start bei 1100 und ende bei 1200. Name der Datei: en_validation_11.csv
Start bei 1200 und ende bei 1300. Name der Datei: en_validation_12.csv
Start bei 1300 und ende bei 1400. Name der Datei: en_validation_13.csv
Start bei 1400 und ende bei 1500. Nam